# Procesamiento de Lenguaje Natural I

**Autor:** Gonzalo G. Fernandez

Clase 2: Word embeddings

## Consigna desafío 2

1. Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
2. Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
3. Graficarlos.
4. Obtener conclusiones.

In [127]:
import re
import string

from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import spacy # python -m spacy download es_core_news_sm
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

## Resolución

El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizarán los diálogos de la película argentina "Nueve Reinas" del año 2000 como corpus.

In [128]:
def parse_srt_to_dialogue(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    blocks = re.split(r"\n\s*\n", content.strip())

    dialogues = []
    for block in blocks:
        lines = block.strip().split("\n")

        # first two lines are number and timestamp
        if len(lines) >= 3:
            dialogue_lines = lines[2:]
            cleaned_lines = [
                re.sub(r"^- ", "", line).strip() for line in dialogue_lines
            ]
            dialogue = " ".join(cleaned_lines).strip()
            if dialogue:
                dialogues.append(dialogue)

    return dialogues


dialogues = parse_srt_to_dialogue("data/nueve_reinas-subtitles.srt")
print(f"Total dialogues extracted: {len(dialogues)}")
for d in dialogues[:5]:
    print(d)

Total dialogues extracted: 1371
¿Qué estás leyendo?
Nada... disculpáme. No, está todo bien.
¿Nada más que esto? Sí.
Esta máquina me vuelve loca. Después lo registro.
1.25 más 3.75 son... 5


In [129]:
df = pd.DataFrame(dialogues, columns=["dialogue"])
df.head()

,dialogue
0,¿Qué estás leyendo?
1,"Nada... disculpáme. No, está todo bien."
2,¿Nada más que esto? Sí.
3,Esta máquina me vuelve loca. Después lo registro.
4,1.25 más 3.75 son... 5


### Preprocesamiento
Transformación de oraciones a secuencia de palabras:

In [130]:
nlp = spacy.load("es_core_news_sm")


def spacy_process(text):
    doc = nlp(text)

    # Tokenization & lemmatization
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)

    # Stop words
    filtered_sentence = []
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)

    # Filter punctuation
    filtered_sentence = [w for w in filtered_sentence if w not in string.punctuation]
    return filtered_sentence

sentence_tokens = [spacy_process(row.iloc[0]) for _, row in df.iterrows()]
print(sentence_tokens[:5])

[['¿', 'leer'], ['...', 'disculpáme'], ['¿'], ['máquina', 'volver', 'loca', 'registro'], ['1.25', '3.75', '...', '5']]


In [131]:
sentence_tokens = [text_to_word_sequence(row.iloc[0]) for _, row in df.iterrows()]
print(sentence_tokens[:5])

[['¿qué', 'estás', 'leyendo'], ['nada', 'disculpáme', 'no', 'está', 'todo', 'bien'], ['¿nada', 'más', 'que', 'esto', 'sí'], ['esta', 'máquina', 'me', 'vuelve', 'loca', 'después', 'lo', 'registro'], ['1', '25', 'más', '3', '75', 'son', '5']]


Se obtuvieron mejores resultados con `text_to_word_sequence` que con `spacy`.

### Creación de los vectores (word2vec)
Añadimos callback en gensim para permitir monitoriar el loss despues de cada epoch:

In [132]:
class callback(CallbackAny2Vec):
    """Callback to print loss after each epoch."""

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print("Loss after epoch {}: {}".format(self.epoch, loss))
        else:
            print(
                "Loss after epoch {}: {}".format(
                    self.epoch, loss - self.loss_previous_step
                )
            )
        self.epoch += 1
        self.loss_previous_step = loss

Creación del modelo generador de vectores, en este caso skipgram:

In [133]:
w2v_model = Word2Vec(
    min_count=5,  # min frquency to include the word in the vocab
    window=2,  # number of words before and after the word
    vector_size=300,  # vector size
    negative=20,  # negative sampling
    workers=4,
    sg=1,  # skipgram (1) or CBOW (0)
)

Obtención del vocabulario a través de los tokens que se obtuvieron en el preprocesamiento:

In [134]:
w2v_model.build_vocab(sentence_tokens)
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 1371
Cantidad de words distintas en el corpus: 326


Entrenamiento del modelo:

In [135]:
w2v_model.train(
    sentence_tokens,
    total_examples=w2v_model.corpus_count,
    epochs=20,
    compute_loss=True,
    callbacks=[callback()],
)

Loss after epoch 0: 82433.296875
Loss after epoch 1: 33194.546875
Loss after epoch 2: 30155.75
Loss after epoch 3: 30698.203125
Loss after epoch 4: 29862.59375
Loss after epoch 5: 30284.5625
Loss after epoch 6: 30285.890625
Loss after epoch 7: 30976.375
Loss after epoch 8: 30267.75
Loss after epoch 9: 29870.34375
Loss after epoch 10: 29932.0
Loss after epoch 11: 29957.96875
Loss after epoch 12: 30697.03125
Loss after epoch 13: 30266.90625
Loss after epoch 14: 31082.46875
Loss after epoch 15: 30619.625
Loss after epoch 16: 31133.625
Loss after epoch 17: 30240.9375
Loss after epoch 18: 31574.6875
Loss after epoch 19: 30371.8125


(82102, 191020)

### Exploración de los vectores

In [136]:
w2v_model.wv.most_similar(positive=["hermano"], topn=10)

[('vieja', 0.9991167187690735),
 ('socio', 0.9990863800048828),
 ('abogado', 0.9990856051445007),
 ('amigo', 0.9990633726119995),
 ('marido', 0.9988681077957153),
 ('viejo', 0.9988553524017334),
 ('mamá', 0.9988453388214111),
 ('valeria', 0.9986215233802795),
 ('tenías', 0.9985424280166626),
 ('pensé', 0.998537003993988)]

Se puede observar como los vectores similares a "hermano" son de palabras que identifican personas.

In [137]:
w2v_model.wv.most_similar(positive=["guita"], topn=10)

[('laburo', 0.9984562993049622),
 ('caja', 0.9984362721443176),
 ('casa', 0.9984316825866699),
 ('plancha', 0.9984221458435059),
 ('podés', 0.998421847820282),
 ('vida', 0.9984095096588135),
 ('aquí', 0.99839848279953),
 ('cifra', 0.9983983039855957),
 ('plata', 0.9983944892883301),
 ('sobre', 0.9983701109886169)]

Se puede observar como los vectores similares a "guita" son de palabras que se relacionan con el dinero (siendo el término "guita" argentino).

In [138]:
w2v_model.wv.most_similar(positive=["cheque"], topn=2)

[('dijiste', 0.9982295036315918), ('dame', 0.9980218410491943)]

Test de analogía con viejo/a y hermano/a:

In [139]:
vector_viejo = w2v_model.wv.get_vector("viejo")
vector_hermano = w2v_model.wv.get_vector("hermano")
vector_vieja = w2v_model.wv.get_vector("vieja")
vector_hermana = w2v_model.wv.get_vector("hermana")

viejo_hermano = vector_viejo - vector_hermano
vieja_hermana = vector_vieja - vector_hermana

cosine_similarity(
    vieja_hermana.reshape(1, -1),
    viejo_hermano.reshape(1, -1),
)[0][0]


-0.15676473

No se obtuvieron buenos resultados en el test de analogía, el valore resultante al analizar similaridad dista de 1.

In [140]:
cosine_similarity(
    w2v_model.wv.get_vector("hoy").reshape(1, -1),
    w2v_model.wv.get_vector("mañana").reshape(1, -1),
)[0][0]

0.9974798

### Visualización
Para poder visualizar el espacio vectorial se realiza una reducción de dimensiones mediante TSNE:

In [141]:
def reduce_dimensions(model, num_dimensions=2):
    """Reduce the dimensions of the word vectors using t-SNE."""
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)
    return vectors, labels

A continuación se visulizan 200 palabras en 2 dimensiones:

In [142]:
MAX_WORDS=200
vecs, labels = reduce_dimensions(w2v_model)
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(render_mode="vscode")

A continuación se visualizan 200 palabras en 3 dimensiones:

In [143]:
vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="vscode")

Con el motivo de poder visualizar los vectores si no se dispone la figura generada por el notebook, se genera un archivo tsv para graficar en http://projector.tensorflow.org/

In [144]:
vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("output/vectors.tsv", vectors, delimiter="\t")

with open("output/labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

## Conclusiones
En la visualización se pueden identificar grupos de interés con gran contenido semántico que exponen el funcionamiento del embedding. Por ejemplo los grupos con las siguientes palabras:
- Marido, hermano, hermana, viejo, vieja, mamá.
- Las, nueve, estampillas
- Que, querés, pasa, pasó, haces
- Mi, tu (practicamente el mismo vector)
Varios insultos se encuentran muy cercanos entre sí.

También la asociación "dame", "dijiste" y "cheque" resulta simpática para quien vio la película.

Se obtuvieron mejores resultados con skipgram que con CBOW. En general, a pesar de experimentar con distintos valores en el modelo, los mejores resultados se obtuvieron con los parámetros utilizados en la práctica de la clase.